In [ ]:
##### PART 1 #####

def MSE(W, b, x, y, reg):
    # reshape y first
    y_reshaped = np.transpose(y)

    # blend the pixel data together per image
    nr_images = x.shape[0]
    x_reshaped = np.reshape(x, (x.shape[0], x.shape[1]*x.shape[2])) # original matrix remains untouched
    x_reshaped = np.transpose(x_reshaped)  

    # term1 = means squared error term
    # term2 = weight decay error term
    term1 = np.dot(np.transpose(W), x_reshaped) + b - y_reshaped # matrix multiplication of 2D matrices
    term1 = np.dot(term1, np.transpose(term1)) / nr_images
    term2 = (reg/2 * np.dot(np.transpose(W), W))
    return (term1 + term2)

def gradMSE(W, b, x, y, reg):
    y_reshaped = np.transpose(y)
    nr_images = x.shape[0]
    x_reshaped = np.reshape(x, (x.shape[0], x.shape[1]*x.shape[2])) # original matrix remains untouched
    x_reshaped = np.transpose(x_reshaped)

    term1 = (np.dot(np.transpose(W), x_reshaped) + b - y_reshaped)
    
    grad_weight = 2/nr_images * np.matmul(x_reshaped, np.transpose(term1)) + reg*W
    grad_bias = 2/nr_images * np.sum(term1)
    
    return grad_weight, grad_bias


def plot_errors(d0, d1, d2, epochs, alpha, reg):
    
    print(f'plot_errors: plotting errors for alpha = {alpha}, reg = {reg}')

    print(f'Max training error = {np.amax(d0)}, Min training error = {np.amin(d0)}')
    print(f'Max val error = {np.amax(d1)}, Min val error = {np.amin(d1)}')
    print(f'Max test error = {np.amax(d2)}, Min test error = {np.amin(d2)}')

    plt.plot(np.arange(epochs), d0, label='Training error')
    plt.plot(np.arange(epochs), d1, label='Validation error')
    plt.plot(np.arange(epochs), d2, label='Testing error')
    plt.title(f'Errors for model trained with alpha = {alpha}, reg = {reg}')
    plt.legend()
    plt.show()

def grad_descent(W, b, x, y, alpha, epochs, reg, error_tol, val_data, val_labels, test_data, test_labels):
    # Your implementation here

    difference_weights = 0
    train_error = []
    val_error = []
    test_error = []

    for t in range(epochs):
        grad_w, grad_b = gradMSE(W, b, x, y, reg)   
        # full data is sent to gradMSE, means all pictures of 28*28
        vt_w = -grad_w  # set direction for weights to move as negative of gradient
        vt_b = -grad_b
        W_new = W + alpha*vt_w  # update the weights
        b_new = b + alpha*vt_b

        single_train_err = MSE(W_new, b_new, x, y, reg)
        single_val_err = MSE(W_new, b_new, val_data, val_labels, reg)
        single_test_err = MSE(W_new, b_new, test_data, test_labels, reg)
        train_error.append(single_train_err)

        val_error.append(single_val_err)
        test_error.append(single_test_err)

        if (np.linalg.norm(W_new - W) < error_tol): # when difference is under error_tol, then done
            W = W_new
            b = b_new
            train_error_array = np.array(train_error)
            val_error_array = np.array(val_error)
            test_error_array = np.array(test_error)
            plot_errors(train_error_array, val_error_array, test_error_array, t+1, alpha, reg)
            return W_new, b_new
        else:
            W = W_new
            b = b_new
		
    train_error_array = np.squeeze(np.array(train_error))
    val_error_array = np.squeeze(np.array(val_error))
    test_error_array = np.squeeze(np.array(test_error))
    plot_errors(train_error_array, val_error_array, test_error_array, epochs, alpha, reg)
    return W, b	# these are updated final

def leastSquares(x, y):
    y_reshaped = np.transpose(y)
    # blend the pixel data together per image
    nr_images = x.shape[0]
    x_reshaped = np.reshape(x, (x.shape[0], x.shape[1]*x.shape[2])) # original matrix remains untouched
    x_pseudo = np.dot(np.linalg.inv(np.dot(np.transpose(x_reshaped), x_reshaped)), np.transpose(x_reshaped))
    W_new = np.dot(x_pseudo, np.transpose(y_reshaped))

    b_new = (1/nr_images)*np.sum(y-np.dot(x_reshaped, W_new))

    return W_new, b_new


In [ ]:
##### PART 2 #####
def crossEntropyLoss(W, b, x, y, reg):
    # reshape x 
    x = x.reshape([x.shape[0], x.shape[1]*x.shape[2]])

    N = y.shape[0]
    
    # calculate y_hat from formula given
    y_hat = 1/(1 + np.exp(-(np.matmul(x,W) + b)))

    # calculate cross entropy loss from formula given
    LD = np.sum((-(y*np.log(y_hat))-((1-y)*np.log(1-y_hat)))/N)
    LW = (reg/2)*(np.linalg.norm(W)**2)
    cross_entropy_loss = LD + LW

    return cross_entropy_loss


def gradCE(W, b, x, y, reg):
    # reshape x 
    x = x.reshape([x.shape[0], x.shape[1]*x.shape[2]])

    x_transpose = x.transpose()
    N = y.shape[0]

    # calculate y_hat from formula given
    y_hat = 1/(1 + np.exp(-(np.matmul(x,W) + b)))

    gradient_w = np.matmul(x_transpose,(y_hat - y))/N + reg*W
    gradient_b = np.sum((y_hat-y)/N)

    return (gradient_w, gradient_b)

def grad_descent(W, b, x, y, alpha, epochs, reg, error_tol, val_data, val_labels, test_data, test_labels, lossType):

    train_error = []
    val_error = []
    test_error = []

    train_accuracy = []
    validity_accuracy = []
    test_accuracy = []
    
    if lossType == "MSE":
      for t in range(epochs):
          grad_w, grad_b = gradMSE(W, b, x, y, reg)   

          vt_w = -grad_w  
          vt_b = -grad_b

          W_new = W + alpha*vt_w  # update the weights
          b_new = b + alpha*vt_b

          single_train_err = MSE(W_new, b_new, x, y, reg)
          single_val_err = MSE(W_new, b_new, val_data, val_labels, reg)
          single_test_err = MSE(W_new, b_new, test_data, test_labels, reg)

          train_error.append(single_train_err)
          val_error.append(single_val_err)
          test_error.append(single_test_err)

          # when difference is under error_tol, then done
          if (np.linalg.norm(W_new - W) < error_tol): 
              W = W_new
              b = b_new
              return W_new, b_new
          else:
              W = W_new
              b = b_new
      
    elif lossType == "CE":
      for t in range(epochs):

          grad_w, grad_b = gradCE(W, b, x, y, reg)   

          vt_w = -grad_w  
          vt_b = -grad_b

          W_new = W + alpha*vt_w  # update the weights
          b_new = b + alpha*vt_b

          single_train_err = crossEntropyLoss(W_new, b_new, x, y, reg)
          single_val_err = crossEntropyLoss(W_new, b_new, val_data, val_labels, reg)
          single_test_err = crossEntropyLoss(W_new, b_new, test_data, test_labels, reg)

          train_error.append(single_train_err)
          val_error.append(single_val_err)
          test_error.append(single_test_err)

          train_acc, val_acc, test_acc = accuracy(W, b, x, y, val_data, val_labels, test_data, test_labels)

          train_accuracy.append(train_acc)
          validity_accuracy.append(val_acc)
          test_accuracy.append(test_acc)

          # when difference is under error_tol, then done
          if (np.linalg.norm(W_new - W) < error_tol): 
              W = W_new
              b = b_new
              train_error_array = np.array(train_error)
              val_error_array = np.array(val_error)
              test_error_array = np.array(test_error)
              plot_errors(train_error_array, val_error_array, test_error_array, t+1, alpha, reg)
              return W_new, b_new
          else:
              W = W_new
              b = b_new

    # plot the loss
    plt.plot(train_error)
    plt.plot(val_error)
    plt.plot(test_error)
    plt.suptitle('Logistic regression Loss: Alpha = %s lamba = %s' %(alpha,reg),fontsize=15)
    plt.legend(['Training Loss','Validation Loss','Test Loss'])
    plt.show()

    # plot the accuracy
    plt.plot(train_accuracy)
    plt.plot(validity_accuracy)
    plt.plot(test_accuracy)
    plt.suptitle('Logistic regression Accuracy: Alpha = %s lamba = %s' %(alpha,reg),fontsize=15)
    plt.legend(['Training Accuracy','Validation Accuracy','Test Accuracy'])
    plt.show()


def accuracy(W, b, x, y, val_data, val_labels, test_data, test_labels):
    
    x = x.reshape([x.shape[0], x.shape[1]*x.shape[2]])
    val_data = val_data.reshape([val_data.shape[0], val_data.shape[1]*val_data.shape[2]])
    test_data = test_data.reshape([test_data.shape[0], test_data.shape[1]*test_data.shape[2]])

    y_hat_train = 1/(1 + np.exp(-(np.matmul(x,W) + b)))
    y_hat_val = 1/(1 + np.exp(-(np.matmul(val_data,W) + b)))
    y_hat_test = 1/(1 + np.exp(-(np.matmul(test_data,W) + b)))

    train_acc = np.sum((y_hat_train>=0.5)==y)/(x.shape[0])
    val_acc = np.sum((y_hat_val>=0.5)==val_labels)/(val_labels.shape[0])
    test_acc = np.sum((y_hat_test>=0.5)==test_labels)/(test_data.shape[0])

    return train_acc, val_acc, test_acc


In [ ]:
##### PART 3 #####

def accuracy (x,W,y):
  W = W.eval()
  y_hat = np.matmul(x,W)

  for i in range(len(y_hat)):
      if (y_hat[i][0])>=0.5: y_hat[i][0]=1
      else: y_hat[i][0] = 0
      
  score = 0
  for i in range(len(y_hat)):
    if y_hat[i][0] == y[i][0]: score+=1

  return score/len(y)

def buildGraph(lossType="MSE", batchSize= 500, betaOne=0.9, betaTwo=0.999, learning_rate=0.001, Epsilon= 1e-08):
  #load the training, validation, and test data and targets
  trainData, validData, testData, trainTarget, validTarget, testTarget = loadData()
  beta = 0
  tf.set_random_seed(421)

  #Initialize weight and bias tensors
  ShapeWVector = (trainData.shape[1]*trainData.shape[2])

  W = tf.Variable(tf.truncated_normal(shape = (ShapeWVector, 1), mean = 0.0, stddev = 0.5, 
                                           dtype = tf.dtypes.float32, seed = None, name = "weight"))
  bias = tf.zeros(shape=(1), dtype=tf.dtypes.float32)
  X = tf.placeholder(dtype = tf.dtypes.float32, shape = (None, ShapeWVector), name = "X")
  Y = tf.placeholder(dtype = tf.dtypes.float32, shape = (None, 1), name = "Y",)
  myLambda = tf.placeholder(dtype = tf.dtypes.float32, shape = None, name = "myLambda")


  if lossType == "MSE":
    prediction = tf.matmul(X, W) + bias
    loss = tf.losses.mean_squared_error(labels = Y, predictions = prediction)
    regularizer = tf.nn.l2_loss(W)
    loss += beta*regularizer

  elif lossType == "CE":
    logit = tf.matmul(X, W) + bias   
    prediction = tf.sigmoid(logit)
    loss = tf.nn.sigmoid_cross_entropy_with_logits(labels = Y, logits = prediction)
    loss = tf.reduce_mean(loss)
    regularizer = tf.nn.l2_loss(W)
    loss += beta*regularizer 

  optimizer = tf.train.AdamOptimizer(learning_rate= learning_rate, beta1= betaOne, beta2= betaTwo, epsilon= Epsilon).minimize(loss)
  
  #Either add SGD code here, or make a new function!

  return W, bias, prediction, Y, X, loss, optimizer

def SGD (lossType="MSE", batchSize= 500, betaOne=0.9, betaTwo=0.999, learning_rate=0.001, Epsilon= 1e-08):

  trainData, validData, testData, trainTarget, validTarget, testTarget = loadData()

  weight, bias, prediction, y, x, loss, optimizer= buildGraph(lossType, batchSize, betaOne, betaTwo, learning_rate, Epsilon)

  trainAccuracy = []
  trainLoss = []

  validAccuracy = []
  validLoss = []

  testAccuracy = []
  testLoss = []

  epochs = 700
  numBatches = int(3500/batchSize)
  validData = np.reshape(validData, (100, 784))
  testData = np.reshape(testData, (145, 784))
  
  with tf.Session() as sess:

    sess.run(tf.global_variables_initializer())
    for j in range(epochs):

      #Shuffling data starts
      temp = np.c_[trainData.reshape(len(trainData), -1), trainTarget.reshape(len(trainTarget), -1)]
      np.random.shuffle(temp)
      trainData = temp[:, :trainData.size//len(trainData)].reshape(trainData.shape)
      trainTarget = temp[:, trainData.size//len(trainData):].reshape(trainTarget.shape)
      #shuffling data ends

      for i in range(numBatches):
        
        data = trainData[i*batchSize: (i+1)*batchSize]
        target = trainTarget[i*batchSize: (i+1)*batchSize]

        data = np.reshape(data,(batchSize, 784))
        target = np.reshape(target, (batchSize, 1))
        sess.run(optimizer, feed_dict = {x:data, y:target})
        
      trainLoss.append(sess.run(loss, feed_dict = {x:data, y:target}))
      validLoss.append(sess.run(loss, feed_dict = {x:validData, y:validTarget}))
      testLoss.append(sess.run(loss, feed_dict = {x:testData, y:testTarget}))
      
      trainAccuracy.append(accuracy(data,weight,target))
      validAccuracy.append(accuracy(validData,weight,validTarget))
      testAccuracy.append(accuracy(testData,weight,testTarget))
      
    #different plotting scales  
    if lossType == "CE":
      LossYlim1 = 0.45
      LossYlim2 = 0.8
      AccuYlim1 = 0.5
      AccuYlim2 = 1.05

    elif lossType == "MSE":
      LossYlim1 = 0
      LossYlim2 = 18
      AccuYlim1 = 0.2
      AccuYlim2 = 1.0

    fig = plt.figure()
    plt.plot(trainAccuracy)
    plt.plot(validAccuracy)
    plt.plot(testAccuracy)
    plt.legend(['Train', 'Valid', 'Test'], loc='lower right')
    plt.xlabel('Epoch', fontsize=16)
    plt.ylabel('Accuracy', fontsize=16)
    axes = plt.gca()
    axes.set_ylim([AccuYlim1, AccuYlim2])
    axes.set_xlim([-50,720])
    plt.show()

    fig = plt.figure()
    plt.plot(trainLoss)
    plt.plot(validLoss)
    plt.plot(testLoss)
    plt.legend(['Train', 'Valid', 'Test'], loc='lower right')
    plt.xlabel('Epoch', fontsize=16)
    plt.ylabel('Loss', fontsize=16)
    axes = plt.gca()
    axes.set_ylim([LossYlim1,LossYlim2])
    axes.set_xlim([-50,720])
    plt.show()  

    print("final training accuracy: ", trainAccuracy[-1])
    print("final valid accuracy: ", validAccuracy[-1])
    print("final test accuracy: ", testAccuracy[-1])